## positive sentiment graph

In [1]:
# 1_network_df

import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
from glob import glob

pd.set_option('display.mpl_style', 'default') 
pd.set_option('display.width', 5000) 
pd.set_option('display.max_columns', 60)

gml_files = glob('../output/network/article_pos1.gml')

/home/gkang/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2885: FutureWarning: 
mpl_style had been deprecated and will be removed in a future version.
Use `matplotlib.pyplot.style.use` instead.

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
def calculate_graph_inf(graph):
    graph.name = filename
    info = nx.info(graph)
    print info
    #plt.figure(figsize=(10,10))
    #nx.draw_spring(graph, arrows=True, with_labels=True)

def highest_centrality(cent_dict):
    """Returns a tuple (node,value) with the node
    with largest value from centrality dictionary."""
    # create ordered tuple of centrality data
    cent_items = [(b,a) for (a,b) in cent_dict.iteritems()]
    # sort in descending order
    cent_items.sort()
    cent_items.reverse()
    return tuple(reversed(cent_items[0]))

In [3]:
network_data_columns = ['name',
                    'sentiment',
                    '# nodes',
                    '# edges',
                    'avg deg',
                    'avg in-deg',
                    'avg out-deg',
                    'density',
                    'deg assort coef', 
                    'avg deg cent',
                    'avg bet cent',
                    'avg clo cent',
                    'high deg cent',
                    'high bet cent',
                    'high clo cent',
                    'avg node conn',
                    '# conn comp',
                    'gc size',
                    '# strong comp',
                    '# weak comp'
                    ]
network_data = pd.DataFrame(columns = network_data_columns)

In [4]:
# graph = directed, ugraph = undirected
for graph_num, gml_graph in enumerate(gml_files):
    graph = nx.read_gml(gml_graph)
    ugraph = graph.to_undirected() # to undirected graph
    U = graph.to_undirected(reciprocal=True)
    e = U.edges()
    ugraph.add_edges_from(e)
    (filepath, filename) = os.path.split(gml_graph)
    print('-' * 10)
    print(gml_graph)
    calculate_graph_inf(graph)
    calculate_graph_inf(ugraph)
    
    # calculate variables
    #sent = filepath.split('/')[-1]
    sent = "positive"
    nodes = nx.number_of_nodes(graph)
    edges = nx.number_of_edges(graph)
    density = float("{0:.4f}".format(nx.density(graph)))
    avg_deg_cen = np.array(nx.degree_centrality(graph).values()).mean()
    avg_bet_cen = np.array(nx.betweenness_centrality(graph).values()).mean()
    avg_clo_cen = np.array(nx.closeness_centrality(graph).values()).mean()
    in_deg = sum(graph.in_degree().values())/float(nx.number_of_nodes(graph))
    out_deg = sum(graph.out_degree().values())/float(nx.number_of_nodes(graph))    
    avg_deg = float("{0:.4f}".format(in_deg + out_deg))
    strong_comp = nx.number_strongly_connected_components(graph)
    weak_comp =  nx.number_weakly_connected_components(graph)
    avg_node_con = float("{0:.4f}".format((nx.average_node_connectivity(graph))))
    deg_assort_coeff = float("{0:.4f}".format((nx.degree_assortativity_coefficient(graph))))
    conn_comp = nx.number_connected_components(ugraph)
    deg_cen = nx.degree_centrality(graph)
    bet_cen = nx.betweenness_centrality(graph)
    clo_cen = nx.closeness_centrality(graph)
    highest_deg_cen = highest_centrality(deg_cen)
    highest_bet_cen = highest_centrality(bet_cen)
    highest_clo_cen = highest_centrality(clo_cen)
    Gc = len(max(nx.connected_component_subgraphs(ugraph), key=len))

    # save variables into list
    graph_values = {'name':filename,
                    'sentiment':sent,
                    '# nodes':nodes,
                    '# edges':edges,
                    'avg deg':avg_deg,
                    'avg in-deg':"%.4f" % in_deg,
                    'avg out-deg':"%.4f" % out_deg,
                    'density':density,
                    'deg assort coef':deg_assort_coeff,
                    'avg deg cent':"%.4f" % avg_deg_cen,
                    'avg bet cent':"%.4f" % avg_bet_cen,
                    'avg clo cent':"%.4f" % avg_clo_cen,
                    'high deg cent':highest_deg_cen,
                    'high bet cent':highest_bet_cen,
                    'high clo cent':highest_clo_cen,
                    'avg node conn':avg_node_con,
                    '# conn comp':conn_comp,
                    'gc size':Gc,
                    '# strong comp':strong_comp,
                    '# weak comp':weak_comp
                    }
    network_data = network_data.append(graph_values, ignore_index=True)

----------
../output/network/article_pos1.gml
Name: article_pos1.gml
Type: MultiDiGraph
Number of nodes: 840
Number of edges: 1127
Average in degree:   1.3417
Average out degree:   1.3417
Name: article_pos1.gml
Type: MultiGraph
Number of nodes: 840
Number of edges: 1127
Average degree:   2.6833


In [5]:
network_data

#network_data.to_csv('output/network_df_.csv', encoding = 'utf-8')

name sentiment  # nodes  # edges  avg deg avg in-deg avg out-deg  density  deg assort coef avg deg cent avg bet cent avg clo cent                high deg cent               high bet cent              high clo cent  avg node conn  # conn comp  gc size  # strong comp  # weak comp
0  article_pos1.gml  positive    840.0   1127.0   2.6833     1.3417      1.3417   0.0016            0.002       0.0032       0.0005       0.0172  (vaccines, 0.0750893921335)  (measles, 0.0466323563202)  (parents, 0.138922123093)         0.0981         58.0    673.0          769.0         58.0

- - -

In [6]:
# 2_node_df
data_columns = ['name',
                'sentiment'
                ]
data = pd.DataFrame(columns = data_columns)
combined_df = pd.DataFrame()

In [7]:
# graph = directed, ugraph = undirected
for graph_num, gml_graph in enumerate(gml_files):
    print('-' * 10)
    print(gml_graph)
    calculate_graph_inf(graph)
    calculate_graph_inf(ugraph)

    # calculate variables and save into list
    #sent = filepath.split('/')[-1]
    deg_cent = nx.degree_centrality(graph)
    bet_cent = nx.betweenness_centrality(graph)
    clo_cent = nx.closeness_centrality(graph)
    graph_values = {'name':filename,
                    'sentiment':sent
                    }
    data = data.append(graph_values, ignore_index=True)

    degree = nx.degree(graph)
    deg_df = pd.DataFrame.from_dict(degree, orient = 'index')
    deg_df.columns = ['degree']
    # degree centrality
    deg_cent = nx.degree_centrality(graph)
    dc_df = pd.DataFrame.from_dict(deg_cent, orient = 'index')
    dc_df.columns = ['deg cent']
    # betweenness centrality
    bet_cent = nx.betweenness_centrality(graph)
    bc_df = pd.DataFrame.from_dict(bet_cent, orient = 'index')
    bc_df.columns = ['bet cent']
    # closeness centrality
    clo_cent = nx.closeness_centrality(graph)
    cc_df = pd.DataFrame.from_dict(clo_cent, orient = 'index')
    cc_df.columns = ['clo cent']
    # concat node frames into node_df
    frames = [deg_df, dc_df, bc_df, cc_df]
    node_df = pd.concat(frames, axis = 1)
    node_df.index.name = 'node'
    node_df = node_df.reset_index()

    values = pd.DataFrame(graph_values, columns = ('name', 'sentiment'), index = [0])
    
    # df = merges graph_values with node_df for single graph and fill NaNs
    df = pd.concat([values, node_df], axis = 1)
    df = df.fillna(method='ffill')
    combined_df = combined_df.append(df)

----------
../output/network/article_pos1.gml
Name: article_pos1.gml
Type: MultiDiGraph
Number of nodes: 840
Number of edges: 1127
Average in degree:   1.3417
Average out degree:   1.3417
Name: article_pos1.gml
Type: MultiGraph
Number of nodes: 840
Number of edges: 1127
Average degree:   2.6833


In [8]:
combined_df

#combined_df.to_csv('.csv', encoding = 'utf-8')

name sentiment                                              node  degree  deg cent      bet cent  clo cent
0    article_pos1.gml  positive                                11-12 years of age       1  0.001192  0.000000e+00  0.000000
1    article_pos1.gml  positive                                   11-18 year olds       1  0.001192  0.000000e+00  0.000000
2    article_pos1.gml  positive                                   16 years of age       1  0.001192  0.000000e+00  0.000000
3    article_pos1.gml  positive       2014-2015 FLULAVAL QUADRIVALENT flu vaccine       7  0.008343  1.564540e-05  0.004768
4    article_pos1.gml  positive                                       Afghanistan       1  0.001192  0.000000e+00  0.000000
5    article_pos1.gml  positive                                     Age of Autism       2  0.002384  0.000000e+00  0.093369
6    article_pos1.gml  positive                      American Medical Association       5  0.005959  0.000000e+00  0.061536
7    article_pos1.gml  positive                                         Americans       1  0.001192  0.000000e+00  0.071466
8    article_pos1.gml  positive                                             Amish       3  0.003576  9.358795e-04  0.074153
9    article_pos1.gml  positive                                  Andrew Wakefield       3  0.003576  0.000000e+00  0.003576
10   article_pos1.gml  positive             Arizona Department of Health Services       2  0.002384  0.000000e+00  0.003311
11   article_pos1.gml  positive                                          Assembly       1  0.001192  0.000000e+00  0.000000
12   article_pos1.gml  positive               Australian social services minister       1  0.001192  0.000000e+00  0.000000
13   article_pos1.gml  positive                                               CDC       4  0.004768  0.000000e+00  0.015572
14   article_pos1.gml  positive                                        California       5  0.005959  1.601717e-03  0.082724
15   article_pos1.gml  positive            California Coalition for Health Choice       3  0.003576  5.006528e-04  0.055456
16   article_pos1.gml  positive                                 California Senate       1  0.001192  0.000000e+00  0.000000
17   article_pos1.gml  positive                               California governor       2  0.002384  0.000000e+00  0.060550
18   article_pos1.gml  positive                                         Caribbean       1  0.001192  0.000000e+00  0.001589
19   article_pos1.gml  positive                                   Catholic Church       1  0.001192  0.000000e+00  0.001192
20   article_pos1.gml  positive                                  Catholic parents       3  0.003576  0.000000e+00  0.004257
21   article_pos1.gml  positive                                         Catholics       2  0.002384  0.000000e+00  0.073223
22   article_pos1.gml  positive                                          Cervarix       3  0.003576  0.000000e+00  0.013152
23   article_pos1.gml  positive                              Christian Scientists       2  0.002384  0.000000e+00  0.002384
24   article_pos1.gml  positive                          Church of the First Born       2  0.002384  0.000000e+00  0.001192
25   article_pos1.gml  positive                                          Colorado       2  0.002384  4.741031e-07  0.001192
26   article_pos1.gml  positive  Department of Public Health Immunization Program       1  0.001192  0.000000e+00  0.001589
27   article_pos1.gml  positive                                        Disneyland       2  0.002384  0.000000e+00  0.000000
28   article_pos1.gml  positive                       Disneyland measles outbreak       8  0.009535  5.091867e-04  0.086943
29   article_pos1.gml  positive                                         Dr. Adams       4  0.004768  0.000000e+00  0.111853
..                ...       ...                                               ...     ...       ...           ...       ...
810  article_pos1.gml  positive              vacc

- - -

In [9]:
# 7_negative_graph_calculation
def drawIt(graph, what = 'graph'):
    nsize = graph.number_of_nodes()
    print "Drawing %s of size %s:" % (what, nsize)
    
    if nsize > 20:
        plt.figure(figsize=(10, 10))
        if nsize > 40:
            nx.draw_spring(graph, with_labels = True, node_size = 70, font_size = 12)
        else:
            nx.draw_spring(graph, with_labels = True)
    else:
        nx.draw_spring(graph, with_labels = True)
    plt.show()

def describeGraph(graph):
    components = sorted(nx.connected_components(graph), key = len, reverse = True)
    cc = [len(c) for c in components]
    subgraphs = list(nx.connected_component_subgraphs(graph))
    params = (graph.number_of_edges(),graph.number_of_nodes(),len(cc))
    print "Graph has %s nodes, %s edges, %s connected components\n" % params
    drawIt(graph)
    for sub in components:
        drawIt(graph.subgraph(sub), what = 'component')

In [10]:
# list of connected components by size
connected_components = [len(c) for c in sorted(nx.connected_components(ugraph), key=len, reverse=True)]

# generate connected components as subgraphs
subgraphs = list(nx.connected_component_subgraphs(ugraph))

# greatest component
Gc = max(nx.connected_component_subgraphs(ugraph), key=len)

# returns all minimum k cutsets of an undirected graph
# i.e., the set(s) of nodes of cardinality equal to the node connectivity of G
# thus if removed, would break G into two or more connected components
cutsets = list(nx.all_node_cuts(Gc))

print "Connected components =", connected_components
print "Greatest component size =", len(Gc)
print "# of cutsets =", len(cutsets)

# returns a set of nodes or edges of minimum cardinality that disconnects G
print "Min node cut =", nx.minimum_node_cut(Gc)
print "Min edge cut =", nx.minimum_edge_cut(Gc)

Connected components = [673, 15, 7, 6, 5, 5, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
Greatest component size = 673
# of cutsets = 147
Min node cut = set([u'threat'])
Min edge cut = set([(u'anti-vaccination', u'time')])


In [11]:
# min cuts with source and target
print nx.minimum_node_cut(Gc, s='vaccines', t='autism')
print nx.minimum_edge_cut(Gc, s='vaccines', t='autism')

set([u'children at higher risk for autism', u'vaccinated children', u'MMR vaccine', u'autism risk', u'high-risk children', u'anti-vaccination', u'parents', u'scientists', u'families', u'children', u'Jain study'])
set([(u'vaccinated high-risk children', u'autism'), (u'genetic predisposition', u'autism'), (u'vaccines', u'autism'), (u'children', u'autism'), (u'MMR vaccine', u'autism'), (u'children at increased genetic risk', u'autism'), (u'scientists', u'autism'), (u'anti-vaccination', u'autism'), (u'families', u'autism'), (u'parents', u'autism'), (u'vaccinated children and unvaccinated children', u'autism'), (u'harmful association', u'autism'), (u'children with affected older sibling', u'autism'), (u'high-risk children', u'autism'), (u'protective effect of vaccines', u'autism'), (u'vaccinated children', u'autism'), (u'children at higher risk for autism', u'autism')])


In [12]:
# read edge labels in min cut for Gc
# change source and target
a = nx.minimum_edge_cut(Gc, s='vaccines', t='autism')
#a = nx.minimum_edge_cut(Gc)

labels = nx.get_edge_attributes(Gc,'edge')
edgelabels = {}
for e in labels.keys():
    e1 = e[0:2]
    edgelabels[e1]=labels[e]

for e in a:
    if edgelabels.has_key(e):
        print e,edgelabels[e]
    else:
        rev_e = e[::-1]
        print rev_e, edgelabels[rev_e]

(u'vaccinated high-risk children', u'autism') are less likely to be diagnosed with
(u'genetic predisposition', u'autism') makes more vulnerable to
(u'vaccines', u'autism') study after study found no link with
(u'children', u'autism') one in 68 kids has some form of
(u'MMR vaccine', u'autism') has nothing to do with
(u'children at increased genetic risk', u'autism') no increased risk even among
(u'scientists', u'autism') remains challenge for
(u'anti-vaccination', u'autism') is driven by fears that shots cause
(u'families', u'autism') remains challenge for
(u'parents', u'autism') who already have a child with autism seem even more concerned
(u'vaccinated children and unvaccinated children', u'autism') age of onset does not differ between
(u'harmful association', u'autism') none between MMR vaccine and
(u'children with affected older sibling', u'autism') 7% themselves had
(u'high-risk children', u'autism') were not more likely to develop
(u'protective effect of vaccines', u'autism') may 

In [13]:
# this takes forever
# average connectivity k of a graph G is the average of local node connectivity over all pairs of nodes of G
#nx.average_node_connectivity(Gc)

- - -

In [14]:
# 8_single_calc_neg
#describeGraph(ugraph)

In [15]:
# degree histogram: returns a list of frequencies of degrees
print "Degree histogram:", nx.degree_histogram(ugraph)

Degree histogram: [0, 476, 151, 71, 43, 28, 20, 7, 6, 9, 5, 3, 0, 1, 1, 0, 3, 0, 2, 0, 1, 0, 2, 1, 0, 0, 1, 1, 2, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [16]:
# degree rank plot (undirected)
degree_sequence = sorted(nx.degree(ugraph).values(),reverse=True) # degree sequence
print "Degree sequence", degree_sequence
dmax = max(degree_sequence)

plt.loglog(degree_sequence,'b-', marker = 'o')
plt.title("Degree rank plot")
plt.ylabel("degree")
plt.xlabel("rank")

# draw graph in inset
plt.axes([0.45,0.45,0.45,0.45])
Gcc = sorted(nx.connected_component_subgraphs(ugraph), key = len, reverse = True)[0]
pos = nx.spring_layout(Gcc)
plt.axis('off')
nx.draw_networkx_nodes(Gcc, pos, node_size = 20)
nx.draw_networkx_edges(Gcc, pos, alpha = 0.4)

plt.show()

Degree sequence [63, 51, 50, 38, 32, 29, 28, 28, 27, 26, 23, 22, 22, 20, 18, 18, 16, 16, 16, 14, 13, 11, 11, 11, 10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 7, 7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

/home/gkang/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:1288: UserWarning: findfont: Font family [u'monospace'] not found. Falling back to Bitstream Vera Sans
  (prop.get_family(), self.defaultFamily[fontext]))


In [17]:
# degree centrality
dc = nx.degree_centrality(graph)
dc_df = pd.DataFrame.from_dict(dc, orient = 'index')
dc_df.columns = ['degree cent']
dc_df = dc_df.sort_values(by = ['degree cent'])
#dc_df

# betweenness centrality
bc = nx.betweenness_centrality(graph)
bc_df = pd.DataFrame.from_dict(bc, orient = 'index')
bc_df.columns = ['betweenness cent']
bc_df = bc_df.sort_values(by = ['betweenness cent'])
#bc_df

# closeness centrality
cc = nx.closeness_centrality(graph)
cc_df = pd.DataFrame.from_dict(cc, orient = 'index')
cc_df.columns = ['closeness cent']
cc_df = cc_df.sort_values(by = ['closeness cent'])
#cc_df

In [18]:
# SUMMARY

#network_data
#combined_df
#dc_df, bc_df, cc_df
print "Degree histogram:", nx.degree_histogram(graph)
print "Connected components =", connected_components
print "Greatest component size =", len(Gc)
#describeGraph(ugraph)
print "# of cutsets =", len(cutsets)
print "Min node cut =", nx.minimum_node_cut(Gc)
print "Min edge cut =", nx.minimum_edge_cut(Gc)

Degree histogram: [0, 476, 151, 71, 43, 28, 20, 7, 6, 9, 5, 3, 0, 1, 1, 0, 3, 0, 2, 0, 1, 0, 2, 1, 0, 0, 1, 1, 2, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
Connected components = [673, 15, 7, 6, 5, 5, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
Greatest component size = 673
# of cutsets = 147
Min node cut = set([u'threat'])
Min edge cut = set([(u'anti-vaccination', u'time')])
